In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Thu Jul  1 07:04:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!unzip -O cp949 '/content/drive/MyDrive/kfood.zip'

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
         continuing with "central" filename version
  inflating: food/족발/Img_131_0673.jpg  
food/족발/Img_131_0674.jpg:  mismatching "local" filename (food/議깅컻/Img_131_0674.jpg),
         continuing with "central" filename version
  inflating: food/족발/Img_131_0674.jpg  
food/족발/Img_131_0675.jpg:  mismatching "local" filename (food/議깅컻/Img_131_0675.jpg),
         continuing with "central" filename version
  inflating: food/족발/Img_131_0675.jpg  
food/족발/Img_131_0676.jpg:  mismatching "local" filename (food/議깅컻/Img_131_0676.jpg),
         continuing with "central" filename version
 extracting: food/족발/Img_131_0676.jpg  
food/족발/Img_131_0677.jpg:  mismatching "local" filename (food/議깅컻/Img_131_0677.jpg),
         continuing with "central" filename version
  inflating: food/족발/Img_131_0677.jpg  
food/족발/Img_131_0678.jpg:  mismatching "local" filename (food/議깅컻/Img_131_0678.jpg),
         continuing with "central" filename version
 extracting: food/족발/Img_13

In [4]:
%%capture
import os
from collections import defaultdict
import shutil, sys 

if not os.path.isdir('/content/test') or not os.path.isdir('/content/train'):

    def copytree(src, dst, symlinks = False, ignore = None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    def generate_dir_file_map(path):
        dir_files = defaultdict(list)
        with open(path, encoding="euc-kr") as txt:
            files = [l.strip() for l in txt.readlines()]
            for f in files:
                dir_name, id = f.split('/')
                dir_files[dir_name].append(id + '.jpg')
        return dir_files

    train_dir_files = generate_dir_file_map('/content/meta/train.txt')
    test_dir_files = generate_dir_file_map('/content/meta/test.txt')


    def ignore_train(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = train_dir_files[subdir]
        return to_ignore

    def ignore_test(d, filenames):
        print(d)
        subdir = d.split('/')[-1]
        to_ignore = test_dir_files[subdir]
        return to_ignore

    copytree('/content/food', '/content/test', ignore=ignore_train)
    copytree('/content/food', '/content/train', ignore=ignore_test)
    
else:
    print('Train/Test files already copied into separate folders.')

In [5]:
os.path.isdir('/content/train')
os.path.isdir('/content/test')

True

In [6]:
from __future__ import print_function
from __future__ import division


from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model
from keras.layers import Dense
from keras.layers import Input
import numpy as np
import argparse

In [7]:
def setup_generator(train_path, test_path, batch_size, dimentions):
    train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_path,  # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    validation_generator = test_datagen.flow_from_directory(
        test_path, # this is the target directory
        target_size=dimentions,
        batch_size=batch_size)

    return train_generator, validation_generator

In [8]:
def load_image(img_path, dimentions, rescale=1. / 255):
    img = image.load_img(img_path, target_size=dimentions)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x *= rescale # rescale the same as when trained

    return x

In [9]:
shape = (224, 224, 3)
X_train, X_test = setup_generator('/content/train', '/content/test', 32, shape[:2])

Found 22632 images belonging to 30 classes.
Found 8095 images belonging to 30 classes.


In [10]:
X_train

In [11]:
X_train.num_classes

30

In [12]:
from keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from keras import layers
NUM_CLASSES = 30

CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'


LOSS_METRICS = ['accuracy']


# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100


In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

In [14]:
import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.layers import Dense, Flatten

base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = AveragePooling2D()(x)

x = Dropout(.5)(x)
x = Flatten()(x)
predictions = Dense(X_train.num_classes, kernel_initializer='glorot_uniform', activation='softmax')(x)

model = Model(base_model.input,predictions)

# 이미 학습된 영역은 학습하지 않겠다고 설정하는 옵션 


2.5.0
94773248/94765736 [==============================] - 1s 0us/step


In [15]:
# 모델 구조
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [16]:
from tensorflow.keras import optimizers

# optimizer, compile
sgd = optimizers.SGD(lr = 0.1,  momentum = 0.9)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau

def schedule(epoch):
    if epoch < 5:
        return 0.001
    elif epoch < 10:
        return .0002
    elif epoch < 15:
        return 0.00002
    else:
        return .0000005
lr_scheduler = LearningRateScheduler(schedule)
csv_logger = CSVLogger('/content/project.log')
checkpointer = ModelCheckpoint(filepath='/content/project.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)

model.fit(X_train, validation_data=X_test,
                              epochs=50,
                              steps_per_epoch=X_train.samples//32,
                              validation_steps=X_test.samples//32,
                               callbacks=[lr_scheduler, csv_logger, checkpointer])

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


532/707 [=====================>........] - ETA: 1:42 - loss: 4.9178 - accuracy: 0.0535

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


707/707 [==============================] - 514s 682ms/step - loss: 4.9452 - accuracy: 0.0559 - val_loss: 4.4375 - val_accuracy: 0.0593

Epoch 00001: val_loss improved from inf to 4.43748, saving model to /content/project.01-4.44.hdf5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
707/707 [==============================] - 456s 644ms/step - loss: 4.6334 - accuracy: 0.0677 - val_loss: 4.6367 - val_accuracy: 0.0681

Epoch 00002: val_loss did not improve from 4.43748
Epoch 3/50
707/707 [==============================] - 446s 631ms/step - loss: 4.5998 - accuracy: 0.0751 - val_loss: 4.5869 - val_accuracy: 0.1097

Epoch 00003: val_loss did not improve from 4.43748
Epoch 4/50
707/707 [==============================] - 444s 628ms/step - loss: 4.7728 - accuracy: 0.0801 - val_loss: 4.3729 - val_accuracy: 0.0930

Epoch 00004: val_loss improved from 4.43748 to 4.37291, saving model to /content/project.04-4.37.hdf5
Epoch 5/50
707/707 [==============================] - 440s 622ms/step - loss: 4.7292 - accuracy: 0.0839 - val_loss: 6.0293 - val_accuracy: 0.1027

Epoch 00005: val_loss did not improve from 4.37291
Epoch 6/50
707/707 [==============================] - 439s 621ms/step - loss: 3.2338 - accuracy: 0.1300 - val_loss: 3.0184 - val_accuracy: 0.1507

Epoch 0000